### Live Prediction

In [67]:
# !pip install opencv-python


In [68]:
import os
import cv2
from ultralytics import YOLO
from pathlib import Path

# Class-color mapping (same as yours)
class_id_to_name = {
    0:  ('road', [28, 42, 168]),
    1:  ('pool', [0, 50, 89]),
    2:  ('vegetation', [107, 142, 35]),
    3:  ('roof', [70, 70, 70]),
    4:  ('wall', [102, 102, 156]),
    5:  ('window', [254, 228, 12]),
    6:  ('person', [255, 22, 96]),
    7:  ('dog', [102, 51, 0]),
    8:  ('car', [9, 143, 150]),
    9:  ('bicycle', [119, 11, 32]),
    10: ('tree', [51, 51, 0]),
    11: ('truck', [160, 160, 60]),
    12: ('bus', [200, 80, 80]),
    13: ('vehicle', [20, 80, 80]),
}

def load_model(model_path="best.pt"):
    return YOLO(model_path)

def predict_frame(model, frame, conf_thresh=0.5, save_dir=None, frame_id="0000"):
    results = model.predict(source=frame, conf=conf_thresh, verbose=False)
    annotated = frame.copy()

    # Prepare directories
    if save_dir:
        os.makedirs(f"{save_dir}/images", exist_ok=True)
        os.makedirs(f"{save_dir}/labels", exist_ok=True)

    label_lines = []
    
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            class_name, color = class_id_to_name.get(cls_id, (f"class_{cls_id}", [0, 255, 0]))
            label = f"{class_name}: {conf:.2f}"

            # Draw on image (annotation)
            cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated, label, (x1, max(y1 - 10, 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            if save_dir:
                # YOLO format: class_id x_center y_center width height (normalized)
                h, w, _ = frame.shape
                xc = (x1 + x2) / 2 / w
                yc = (y1 + y2) / 2 / h
                bw = (x2 - x1) / w
                bh = (y2 - y1) / h
                label_lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

    if save_dir:
        # Save original image (real image)
        cv2.imwrite(f"{save_dir}/images/frame_{frame_id}.jpg", frame)
        # Save annotated image
        # cv2.imwrite(f"{save_dir}/images/frame_{frame_id}_annotated.jpg", annotated)
        # Save label in YOLO format
        with open(f"{save_dir}/labels/frame_{frame_id}.txt", "w") as f:
            f.write("\n".join(label_lines))

    return annotated

def run_webcam_detection(model_path="best.pt", save=False):
    model = load_model(model_path)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    save_dir = "output" if save else None
    frame_count = 0

    print("Press 'q' to exit...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id = f"{frame_count:05d}"
        annotated_frame = predict_frame(model, frame, save_dir=save_dir, frame_id=frame_id)
        cv2.imshow("YOLOv8 Live Detection", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()



In [69]:
def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"✅ Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


In [70]:
import os
import random
import cv2
import numpy as np

# Function to draw bounding boxes from YOLO format labels
def draw_bounding_boxes(image, label_path, class_id_to_name):
    with open(label_path, "r") as file:
        labels = file.readlines()

    for label in labels:
        parts = label.strip().split()
        class_id = int(parts[0])
        xc, yc, bw, bh = map(float, parts[1:])
        
        # Get the image dimensions
        h, w, _ = image.shape

        # Convert YOLO format to pixel values
        x1 = int((xc - bw / 2) * w)
        y1 = int((yc - bh / 2) * h)
        x2 = int((xc + bw / 2) * w)
        y2 = int((yc + bh / 2) * h)

        # Get class name and color
        class_name, color = class_id_to_name.get(class_id, (f"class_{class_id}", [0, 255, 0]))

        # Draw the bounding box and label
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f"{class_name}", (x1, max(y1 - 10, 10)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    return image

def visualize_random_images(images_dir="./datasets/split_videos_dataset/train/images", 
                            labels_dir="./datasets/split_videos_dataset/train/labels", 
                            num_images=10, 
                            max_display_size=800):
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')]
    random_images = random.sample(image_files, min(num_images, len(image_files)))

    for image_file in random_images:
        image_path = os.path.join(images_dir, image_file)
        label_path = os.path.join(labels_dir, f"{os.path.splitext(image_file)[0]}.txt")
        
        if not os.path.exists(label_path):
            print(f"Warning: No label file found for {image_path}")
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Warning: Failed to read {image_path}")
            continue

        print("Image:", image_path)
        image_with_boxes = draw_bounding_boxes(image, label_path, class_id_to_name)

        # Resize image to fit screen
        h, w = image_with_boxes.shape[:2]
        scale = min(max_display_size / max(h, w), 1.0)
        resized = cv2.resize(image_with_boxes, (int(w * scale), int(h * scale)))

        # Display image
        cv2.imshow(f"Image: {image_file}", resized)
        cv2.waitKey(0)

    cv2.destroyAllWindows()


In [71]:
import os
import cv2
from ultralytics import YOLO
from pathlib import Path

# Class-color mapping (same as yours)
class_id_to_name = {
    0:  ('unlabeled', [28, 42, 168]),
    1:  ('wall', [0, 50, 89]),
    2:  ('vegetation', [107, 142, 35]),
    3:  ('roof', [70, 70, 70]),
    4:  ('wall', [102, 102, 156]),
    5:  ('window', [254, 228, 12]),
    6:  ('person', [255, 22, 96]),
    7:  ('dog', [102, 51, 0]),
    8:  ('car', [9, 143, 150]),
    9:  ('bicycle', [119, 11, 32]),
    10: ('tree', [51, 51, 0]),
    11: ('truck', [160, 160, 60]),
    12: ('bus', [200, 80, 80]),
    13: ('vehicle', [20, 80, 80]),
}

def load_model(model_path="best.pt"):
    return YOLO(model_path)

def predict_frame(model, frame, conf_thresh=0.3, save_dir=None, frame_id="0000"):
    results = model.predict(source=frame, conf=conf_thresh, verbose=False)
    annotated = frame.copy()

    # Prepare directories
    if save_dir:
        os.makedirs(f"{save_dir}/images", exist_ok=True)
        os.makedirs(f"{save_dir}/labels", exist_ok=True)

    label_lines = []
    
    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            class_name, color = class_id_to_name.get(cls_id, (f"class_{cls_id}", [0, 255, 0]))
            label = f"{class_name}: {conf:.2f}"

            # Draw on image (annotation)
            cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated, label, (x1, max(y1 - 10, 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            if save_dir:
                # YOLO format: class_id x_center y_center width height (normalized)
                h, w, _ = frame.shape
                xc = (x1 + x2) / 2 / w
                yc = (y1 + y2) / 2 / h
                bw = (x2 - x1) / w
                bh = (y2 - y1) / h
                label_lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

    if save_dir:
        # Save original image (real image)
        cv2.imwrite(f"{save_dir}/images/frame_{frame_id}.jpg", frame)
        # Save label in YOLO format
        with open(f"{save_dir}/labels/frame_{frame_id}.txt", "w") as f:
            f.write("\n".join(label_lines))

    return annotated

def compare_two_models_live(model_a_path="best.pt", model_b_path="best.pt", save=False):
    # Load two models
    model_a = load_model(model_a_path)
    model_b = load_model(model_b_path)
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    save_dir = "output" if save else None
    frame_count = 0

    print("Press 'q' to exit...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_id = f"{frame_count:05d}"

        # Annotate frames using both models
        left_frame = predict_frame(model_a, frame, save_dir=save_dir, frame_id=frame_id)
        right_frame = predict_frame(model_b, frame, save_dir=save_dir, frame_id=frame_id)

        # Resize frames to a consistent height while maintaining the aspect ratio
        target_height = 420  # Target height for resizing
        left_resized = resize_to_height(left_frame, target_height)
        right_resized = resize_to_height(right_frame, target_height)

        # Find the maximum width between the two resized frames to align them horizontally
        combined_width = left_resized.shape[1] + right_resized.shape[1]
        combined_frame = cv2.hconcat([left_resized, right_resized])

        # Display the combined output of both models
        cv2.imshow("Compare: Model A (Left) vs Model B (Right)", combined_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

def resize_to_height(image, target_height=420):
    """
    Resize image while maintaining aspect ratio.
    """
    h, w = image.shape[:2]
    scale = target_height / h
    new_w = int(w * scale)
    return cv2.resize(image, (new_w, target_height))



In [72]:
# import cv2

# def check_cameras():
#     # Try accessing different camera indices, typically 0 is the default (laptop camera)
#     for i in range(5):  # You can increase the range if you have more cameras
#         cap = cv2.VideoCapture(i)
#         if cap.isOpened():
#             print(f"Camera {i} is accessible.")
#             ret, frame = cap.read()
#             if ret:
#                 print(f"Camera {i} is showing a frame.")
#                 cv2.imshow(f"Camera {i}", frame)
#                 cv2.waitKey(0)  # Wait for any key to close the window
#             cap.release()
#         else:
#             print(f"Camera {i} is not accessible.")

#     cv2.destroyAllWindows()

# # Call the function to check for cameras
# check_cameras()


In [73]:
yolov8 = './runs/train/yolov8'
yolov8_retrain = './runs/train/fine-tune-yolov8'

best_pt_path = find_best_model(yolov8)
best_pt_path_retrain = find_best_model(yolov8_retrain)

# run_webcam_detection(best_pt_path, True)

# Run webcam detection with two models
compare_two_models_live(model_a_path=best_pt_path, model_b_path=best_pt_path_retrain, save=False)


✅ Found best.pt at: runs\train\yolov8\weights\best.pt
✅ Found best.pt at: runs\train\fine-tune-yolov8\weights\best.pt
Press 'q' to exit...


In [74]:

# # Function to visualize random images and bounding boxes
# visualize_random_images(images_dir="./datasets/split_videos_dataset/train/images", 
#                             labels_dir="./datasets/split_videos_dataset/train/labels", 
#                             num_images=20, 
#                             max_display_size=720)

In [75]:
import os

def check_image_label_correspondence(images_dir, labels_dir, image_exts={'.jpg', '.png'}, label_ext='.txt'):
    image_files = [f for f in os.listdir(images_dir) if os.path.splitext(f)[1].lower() in image_exts]
    label_files = [f for f in os.listdir(labels_dir) if f.endswith(label_ext)]

    image_basenames = set(os.path.splitext(f)[0] for f in image_files)
    label_basenames = set(os.path.splitext(f)[0] for f in label_files)

    missing_labels = image_basenames - label_basenames
    missing_images = label_basenames - image_basenames

    if missing_labels:
        print("❌ Missing labels for the following images:")
        for name in sorted(missing_labels):
            print(f"- {name}")

    if missing_images:
        print("❌ Missing images for the following labels:")
        for name in sorted(missing_images):
            print(f"- {name}")

    if not missing_labels and not missing_images:
        print("✅ All images have corresponding labels and vice versa!")

# ======== Example usage ========
images_path = "./datasets/split_videos_dataset/train/images"
labels_path = "./datasets/split_videos_dataset/train/labels"
check_image_label_correspondence(images_path, labels_path)


✅ All images have corresponding labels and vice versa!
